# COGS 118B Final Project
## Rhythm Game Beat Chart Difficulty Classification

**Contributers:** Miles Davis, Theo Bui, Alan De Luna, Sanjith Shanmugavel, Nicole Huynh

#### Introduction

Rhythm games are blah blah blah. Add relevant background and motivation for classifying difficulties. Maybe something along the lines of: "Difficulty levels between songs in ___ game are inconsistent. Some beat charts may be considered more difficult because of the number of required hits for a high score simply increasing, increasing tempo, or rhythms becoming more complex."

#### Algorithm Choice
???

#### Data Preprocessing
1. read in `.tja` files as string
2. convert string to `numpy` array
3. extract relevant information:
- `COURSE`
- `LEVEL`
- `BALLOON`
- song notation -- written between `#START` and `#END` commands
- timing notation -- `#MEASURE` and `#BPMCHANGE`
    - from Taiko explanation: 'Measures in the chart are separated with a comma character , followed by a line break. Timing between each measure is the same as long as #MEASURE and #BPMCHANGE commands are not used. Measures may contain any amount of notes, including zero, the less numbers there are in a measure, the more far apart the notes will be in the chart, each measure is equally divided by the amount of numbers there are inside. "12," can be written as "1020," and "10002000,", the timing is identical in all three examples.'
- delay -- offsets position of following notation (can be negative to indicate overlap) and written as `#DELAY`
- scroll speed -- `#SCROLL`
- Go-Go Time -- written between `#GOGOSTART` and `#GOGOEND`
- measure lines turned on/off -- `#BARLINEOFF` and `#BARLINEON`
- branching and paths -- ???
- BPM change -- ???